## Download Google street view images of all cycling accident points

In [ ]:
import os
import requests
import csv
import pandas as pd
import math

In [ ]:
df = pd.read_csv('Cycling/GOV_Selected_2/06_Cycle_All Accident_Accident Rate.csv', encoding='utf-8-sig')
latitudes = df['latitude'].tolist()
longitudes = df['longitude'].tolist()
accident_reference = df['accident_reference_x'].tolist()

print(latitudes)

In [ ]:
print(longitudes)

In [ ]:
# Calculate the function of direction
def calculate_bearing(pointA, pointB):
    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2() returns values from -π to + π
    # So we normalize the result, like so:
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

# Calculate the orientation of each point
bearings = []
for i in range(1, len(latitudes)):
    bearings.append(calculate_bearing((float(latitudes[i-1]), float(longitudes[i-1])), (float(latitudes[i]), float(longitudes[i]))))

# For each point, download the street view facing that direction
# replace ‘Google API Key’ with your own Google API Key Number
for i in range(1, len(latitudes)):
    for angle in range(0, 360, 90):
        ###-45, 315, 90
        ### 0, 360, 90
        adjusted_bearing = (bearings[i-1] + angle) % 360
        # Generate URL
        url = "https://maps.googleapis.com/maps/api/streetview?size=1024x1024&location=" \
              + str(latitudes[i]) + "," + str(longitudes[i]) \
              + "&fov=90&heading=" + str(adjusted_bearing) \
              + "&pitch=0" + "&key=Google API Key"
            
         # Download images
        download_folder = os.path.join(os.getcwd(), 'Streetview Downloads')
        os.makedirs(download_folder, exist_ok=True)
        r = requests.get(url)
        with open(os.path.join(download_folder, str(accident_reference[i]) + "_" + str(int(adjusted_bearing)) + '.jpg'), 'wb') as f:
            f.write(r.content)

print("All Street View Images download is complete!")